In [2]:
import numpy as np
import pdb

# Suppose k,k',l are fixed

# Example Usage
X = np.random.uniform(size=1000)  # Example of scores(X_i, k) for i in D_l
Y = np.random.uniform(size=1000)  # Example of scores(X_i, k') for i in D_l

## Naive approach

In [3]:
def empirical_cdf_2d_naive(X, Y, x_query, y_query):
    """
    Naive implementation of 2D empirical CDF.
    
    Parameters:
    X (np.array): 1D array of observed values for the first random variable
    Y (np.array): 1D array of observed values for the second random variable (same length)
    x_query (float): The x-coordinate where the CDF is evaluated
    y_query (float): The y-coordinate where the CDF is evaluated
    
    Returns:
    float: The value of the empirical CDF at point (x_query, y_query)
    """
    assert len(X) == len(Y)
    count = 0
    total_points = len(X)
    
    # Count how many points (X_i, Y_i) are <= (x_query, y_query)
    for i in range(total_points):
        if X[i] <= x_query and Y[i] <= y_query:
            count += 1
    
    # Empirical CDF is the proportion of points satisfying the condition
    return count / total_points

# Example usage

# Define the range of query points (e.g., 10 different points in x and y)
grid_size = 100
x_queries = np.linspace(0, 1, grid_size)  # 10 points between -2 and 2 for x
y_queries = np.linspace(0, 1, grid_size)  # 10 points between -2 and 2 for y

# Create an empty array to store the results
results_naive = np.zeros((len(x_queries), len(y_queries)))

# Evaluate the empirical CDF at each combination of x_query and y_query, and store the result
for i, x in enumerate(x_queries):
    for j, y in enumerate(y_queries):
        results_naive[i, j] = empirical_cdf_2d_naive(X, Y, x, y)

# Print the result array
print("Empirical CDF results (naive):")
print(results_naive)

Empirical CDF results (naive):
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.008 0.008 0.008]
 [0.    0.    0.    ... 0.02  0.021 0.021]
 ...
 [0.    0.01  0.021 ... 0.946 0.958 0.974]
 [0.    0.012 0.023 ... 0.958 0.97  0.986]
 [0.    0.012 0.023 ... 0.972 0.984 1.   ]]


## Less naive approach

In [4]:
import numpy as np

class EmpiricalCDF2D:
    def __init__(self, X, Y):
        """
        Initialize the 2D empirical CDF by sorting the (X, Y) pairs together.

        Parameters:
        X (np.array): 1D array of observed values for the first random variable
        Y (np.array): 1D array of observed values for the second random variable
        """
        self.n = len(X)
        
        # Combine X and Y into pairs and sort by X first, then Y (lexicographically)
        sorted_indices = np.lexsort((Y, X))
        self.X_sorted = X[sorted_indices]
        self.Y_sorted = Y[sorted_indices]

    def query(self, x, y):
        """
        Query the empirical CDF at point (x, y) using precomputed sorted data.

        Parameters:
        x (float): The x-coordinate where the CDF is evaluated
        y (float): The y-coordinate where the CDF is evaluated
        
        Returns:
        float: The value of the empirical CDF at point (x, y)
        """
        # Binary search to find how many X values are <= x
        idx_x = np.searchsorted(self.X_sorted, x, side='right')
        
        if idx_x == 0:
            return 0.0  # If no points have X <= x, return 0
        
        # Find how many Y values (among the first idx_x points) are <= y
        count_y = np.sum(self.Y_sorted[:idx_x] <= y)
        
        # Return the proportion of points satisfying the condition
        return count_y / self.n

In [5]:
# Initialize the 2D empirical CDF with the observed data and grid
ecdf_2d_grid = EmpiricalCDF2D(X, Y)

In [6]:
# Create an empty array to store the results
results_new = np.zeros((len(x_queries), len(y_queries)))

# Evaluate the empirical CDF at each combination of x_query and y_query, and store the result
for i, x in enumerate(x_queries):
    for j, y in enumerate(y_queries):
        results_new[i, j] = ecdf_2d_grid.query(x, y)

# Print the result array
print("Empirical CDF results (new):")
print(results_new)

print("Empirical CDF results (naive):")
print(results_naive)

print("Difference between new and naive:")
print(results_new-results_naive)

Empirical CDF results (new):
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.008 0.008 0.008]
 [0.    0.    0.    ... 0.02  0.021 0.021]
 ...
 [0.    0.01  0.021 ... 0.946 0.958 0.974]
 [0.    0.012 0.023 ... 0.958 0.97  0.986]
 [0.    0.012 0.023 ... 0.972 0.984 1.   ]]
Empirical CDF results (naive):
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.008 0.008 0.008]
 [0.    0.    0.    ... 0.02  0.021 0.021]
 ...
 [0.    0.01  0.021 ... 0.946 0.958 0.974]
 [0.    0.012 0.023 ... 0.958 0.97  0.986]
 [0.    0.012 0.023 ... 0.972 0.984 1.   ]]
Difference between new and naive:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Third approach

In [7]:
class EmpiricalCDF2DGrid:
    def __init__(self, X, Y):
        """
        Initialize the 2D empirical CDF by sorting the (X, Y) pairs together.

        Parameters:
        X (np.array): 1D array of observed values for the first random variable
        Y (np.array): 1D array of observed values for the second random variable
        """
        self.n = len(X)
        # Combine X and Y into pairs and sort by X first, then Y (lexicographically)
        sorted_indices = np.lexsort((Y, X))
        self.X_sorted = X[sorted_indices]
        self.Y_sorted = Y[sorted_indices]
        self.grid_evaluated = False

    def evaluate_grid(self, x_grid, y_grid):
        """
        Evaluate the empirical CDF at each point on the provided grid of (x, y) values.

        Parameters:
        x_grid (np.array): 1D array of x-coordinates where the CDF is evaluated
        y_grid (np.array): 1D array of y-coordinates where the CDF is evaluated
        
        Returns:
        np.array: A 2D array of ECDF values for each (x, y) combination on the grid
        """
        # Store the grid for later querying
        self.x_grid = x_grid
        self.y_grid = y_grid

        # Precompute positions of x_grid and y_grid in the sorted data
        x_positions = np.searchsorted(self.X_sorted, x_grid, side='right')
        cdf_grid = np.zeros((len(x_grid), len(y_grid)))

        for i, x_pos in enumerate(x_positions):
            if x_pos == 0:
                cdf_grid[i, :] = 0.0  # If no points have X <= x_grid[i], the ECDF is 0
            else:
                # Precompute Y counts up to x_pos for the entire y_grid
                y_counts = np.searchsorted(np.sort(self.Y_sorted[:x_pos]), y_grid, side='right')
                cdf_grid[i, :] = y_counts / self.n

        # Mark that the grid has been evaluated
        self.grid_evaluated = True
        self.cdf_grid = cdf_grid

        return cdf_grid

    def query(self, x, y):
        """
        Query the empirical CDF at point (x, y). If the point is on the precomputed grid,
        return the exact grid result; otherwise, calculate using the search method.

        Parameters:
        x (float): The x-coordinate where the CDF is evaluated
        y (float): The y-coordinate where the CDF is evaluated
        
        Returns:
        float: The value of the empirical CDF at point (x, y)
        """
        # Check if the grid has been evaluated
        if self.grid_evaluated:
            # Try to find the exact match in the grid
            x_idx = np.searchsorted(self.x_grid, x, side='left')
            y_idx = np.searchsorted(self.y_grid, y, side='left')

            # If the point belongs to the grid, return the grid value
            if (x_idx < len(self.x_grid) and y_idx < len(self.y_grid) and
                np.isclose(self.x_grid[x_idx], x) and np.isclose(self.y_grid[y_idx], y)):
                return self.cdf_grid[x_idx, y_idx]

        # If the point is not in the grid, compute it directly using search
        idx_x = np.searchsorted(self.X_sorted, x, side='right')
        if idx_x == 0:
            return 0.0
        count_y = np.sum(self.Y_sorted[:idx_x] <= y)
        return count_y / self.n

# Example Usage

# Initialize the 2D empirical CDF with the observed data
ecdf_2d_grid = EmpiricalCDF2DGrid(X, Y)

# Define the grid where you want to evaluate the CDF
x_grid = np.linspace(0, 1, 100)  # 100 points between -3 and 3 for x
y_grid = np.linspace(0, 1, 100)  # 100 points between -3 and 3 for y

# Evaluate the ECDF on the grid
results_grid = ecdf_2d_grid.evaluate_grid(x_grid, y_grid)

# Test the query function for a grid point and a non-grid point
grid_query_result = ecdf_2d_grid.query(0.0, 0.0)  # On grid
non_grid_query_result = ecdf_2d_grid.query(0.15, 0.15)  # Off grid

# Print the result
print(f"Grid query result: {grid_query_result}")
print(f"Non-grid query result: {non_grid_query_result}")

Grid query result: 0.0
Non-grid query result: 0.025


In [9]:
# Create an empty array to store the results
results_3 = np.zeros((len(x_queries), len(y_queries)))

ecdf_2d_grid = EmpiricalCDF2DGrid(X, Y)

# Evaluate the ECDF on the grid
results_3 = ecdf_2d_grid.evaluate_grid(x_queries, y_queries)

# Print the result array
print("Empirical CDF results (new):")
print(results_3)

print("Empirical CDF results (naive):")
print(results_naive)

print("Difference between new and naive:")
print(results_3-results_naive)

Empirical CDF results (new):
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.008 0.008 0.008]
 [0.    0.    0.    ... 0.02  0.021 0.021]
 ...
 [0.    0.01  0.021 ... 0.946 0.958 0.974]
 [0.    0.012 0.023 ... 0.958 0.97  0.986]
 [0.    0.012 0.023 ... 0.972 0.984 1.   ]]
Empirical CDF results (naive):
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.008 0.008 0.008]
 [0.    0.    0.    ... 0.02  0.021 0.021]
 ...
 [0.    0.01  0.021 ... 0.946 0.958 0.974]
 [0.    0.012 0.023 ... 0.958 0.97  0.986]
 [0.    0.012 0.023 ... 0.972 0.984 1.   ]]
Difference between new and naive:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Method comparison

In [10]:
import time

# Function to compare both methods
def compare_ecdf_methods(X, Y, x_queries, y_queries):
    # Timing the naive approach
    start_time_naive = time.time()
    results_naive = np.zeros((len(x_queries), len(y_queries)))
    for i, x in enumerate(x_queries):
        for j, y in enumerate(y_queries):
            results_naive[i, j] = empirical_cdf_2d_naive(X, Y, x, y)
    end_time_naive = time.time()
    naive_time = end_time_naive - start_time_naive

    # Timing the second approach
    ecdf_2d = EmpiricalCDF2D(X, Y)
    start_time_2 = time.time()
    results_2 = np.zeros((len(x_queries), len(y_queries)))
    for i, x in enumerate(x_queries):
        for j, y in enumerate(y_queries):
            results_2[i, j] = ecdf_2d.query(x, y)
    end_time_2 = time.time()
    time_2 = end_time_2 - start_time_2
    
    # Timing the third approach
    ecdf_2d_grid = EmpiricalCDF2DGrid(X, Y)
    start_time_3 = time.time()
    results_3 = ecdf_2d_grid.evaluate_grid(x_queries, y_queries)
    end_time_3 = time.time()
    time_3 = end_time_3 - start_time_3

    # Check if the results are identical
    identical_2 = np.allclose(results_naive, results_2)
    identical_3 = np.allclose(results_naive, results_3)

    # Print out the results
    print(f"Naive approach time: {naive_time:.6f} seconds")
    print(f"Second approach time: {time_2:.6f} seconds")
    print(f"Results identical: {identical_2}")
    print(f"Third approach time: {time_3:.6f} seconds")
    print(f"Results identical: {identical_3}")

    return results_naive, results_2, identical_2, results_3, identical_3

In [18]:
sample_size = 100
grid_size = 400

# Example Usage
X = np.random.uniform(size=sample_size)  # Example of scores(X_i, k) for i in D_l
Y = np.random.uniform(size=sample_size)  # Example of scores(X_i, k') for i in D_l

# Define the range of query points (e.g., 10 different points in x and y)
x_queries = np.linspace(0, 1, grid_size)  # 10 points between -2 and 2 for x
y_queries = np.linspace(0, 1, grid_size)  # 10 points between -2 and 2 for y

# Compare the methods and print the results
_ = compare_ecdf_methods(X, Y, x_queries, y_queries)

Naive approach time: 3.227436 seconds
Second approach time: 1.554780 seconds
Results identical: True
Third approach time: 0.005134 seconds
Results identical: True


In [19]:
3.227436 / 0.005134

628.6396571873782